In [9]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.api import Logit

In [10]:
df = pd.read_csv("bill_data.csv", sep = ";")
test = pd.read_csv("billets_production.csv")

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   diagonal      5 non-null      float64
 1   height_left   5 non-null      float64
 2   height_right  5 non-null      float64
 3   margin_low    5 non-null      float64
 4   margin_up     5 non-null      float64
 5   length        5 non-null      float64
 6   id            5 non-null      object 
dtypes: float64(6), object(1)
memory usage: 412.0+ bytes


In [12]:
# Train model
y_bill = df.loc[:, df.columns == "is_genuine"]

X_bill = df[["margin_low", "margin_up", "length"]]
X_bill = sm.add_constant(X_bill)

In [13]:
log_reg = Logit(endog = y_bill, exog = X_bill)
log_reg_model = log_reg.fit()

Optimization terminated successfully.
         Current function value: 0.031729
         Iterations 12


In [14]:
X_test = test[["margin_low", "margin_up", "length"]]
X_test = sm.add_constant(X_test)

In [15]:
test["Probabilité d'authenticité"] = log_reg_model.predict(X_test)
test["Probabilité d'authenticité"] = test["Probabilité d'authenticité"] *100
test["Probabilité d'authenticité"] = round(test["Probabilité d'authenticité"],2)
test["Probabilité d'authenticité"] = test["Probabilité d'authenticité"].astype(str) + "%"
test["Prédiction"] = (log_reg_model.predict(X_test) >= 0.5).astype(int)
print(test[["id", "Probabilité d'authenticité", "Prédiction"]])

    id Probabilité d'authenticité  Prédiction
0  A_1                       0.0%           0
1  A_2                       0.0%           0
2  A_3                       0.0%           0
3  A_4                     99.91%           1
4  A_5                     100.0%           1


In [16]:
print("Authentification des billets : \n")
for i, j in zip(test["Prédiction"], test["id"]) :
    if i == 1 :
        print("Le billet", "{}".format(j), "est authentique")
    else :
        print("Le billet", "{}".format(j), "est contrefait")

Authentification des billets : 

Le billet A_1 est contrefait
Le billet A_2 est contrefait
Le billet A_3 est contrefait
Le billet A_4 est authentique
Le billet A_5 est authentique
